This notebook is aiming at prepare training data for third classifier for lower group of multi-step strategies. It also contains the part to kick off training jobs after data is prepared. (Training job will use "auto-split" mode to reserve validation data). Also, this training set includes the keyword records prepared in other notebook.

Prerequisite:

1) csv files containing extracted text files

2) A csv file prepared in Step2 notebook, containing label information corresponding to text files

3) Keyword records prepared in step2.5 notebook

Output:

1) Prepared training file for lower group classifier

2) Kick off binary classification training job

In [1]:
# Import required libraries
import pandas as pd
import numpy as np
import boto3

In [2]:
# Load prior prepared extracted text files
textract_df = pd.read_csv("Textract_output_train04.csv")
textract_df.rename(columns={'file_name' : 'filename'}, inplace=True)
textract_df

filename  page  \
0                 1003700000000530903_11159719.pdf     1   
1                 1003700000000530903_11159719.pdf     2   
2                 1003700000000530903_11159719.pdf     3   
3                 1003700000000530903_11159719.pdf     4   
4                 1003700000000530903_11159719.pdf     5   
...                                            ...   ...   
840163  OCR_33049_1003700000000697103_11103804.pdf    43   
840164  OCR_33049_1003700000000697103_11103804.pdf    44   
840165  OCR_33049_1003700000000697103_11103804.pdf    45   
840166  OCR_33049_1003700000000697103_11103804.pdf    46   
840167  OCR_33049_1003700000000697103_11103804.pdf    47   

                                                  content  
0       PATIENT NAME : BAKER, EDWINNA TLEXINGTON, KENT...  
1       UKHealthCareBAKER, EDWINNADOS: 04/15/2018DOB: ...  
2       DOS: 04/15/2018UKHealthCareBAKER, EDWINNADOB: ...  
3       UK HealthCareBAKER, EDWINNADOS: 04/15/2018DOB:...  
4       OF016785388 8105DOS 04/15/2018Commonwealth of ...  
...                                                   ...  
840163  Patient: GABRY, KAREN11800 E. Twelve Mile Road...  
840164  Patient:GABRY, KAREN11800 E. Twelve Mile RoadA...  
840165  Patient:GABRY, KAREN11800 E. Twelve Mile RoadA...  
840166  Patient:GABRY, KAREN11800 E. Twelve Mile RoadA...  
840167  Patient:GABRY, KAREN11800 E. Twelve Mile RoadA...  

[840168 rows x 3 columns]

In [3]:
# Change filename to same format 
textract_df['filename'] = textract_df['filename'].str.split('_').str[-2] + '_' + textract_df['filename'].str.split('_').str[-1]

In [4]:
# Load prior prepared reformat label file
label_df = pd.read_csv("batch3_label_revised.csv")
label_df

page             class                          filename  filename_cut
0         29   admission_order  1003700000000633142_11068501.pdf        633142
1         43  history_physical  1003700000000633142_11068501.pdf        633142
2         44  history_physical  1003700000000633142_11068501.pdf        633142
3         45  history_physical  1003700000000633142_11068501.pdf        633142
4         46  history_physical  1003700000000633142_11068501.pdf        633142
...      ...               ...                               ...           ...
357778    81  history_physical  1003700000000675009_11251999.pdf        675009
357779    82  history_physical  1003700000000675009_11251999.pdf        675009
357780    83  history_physical  1003700000000675009_11251999.pdf        675009
357781    84  history_physical  1003700000000675009_11251999.pdf        675009
357782    85  history_physical  1003700000000675009_11251999.pdf        675009

[357783 rows x 4 columns]

In [5]:
# In the label file, only keeping the labels for PR/PQ/DD/CM/NA classes
new_label_df = pd.DataFrame()
pr_df = label_df[label_df['class']=='procedures']
pq_df = label_df[label_df['class']=='physician_query']
dd_df = label_df[label_df['class']=='discharge_disposition']
ca_df = label_df[label_df['class']=='case_management']
na_df = label_df[label_df['class']=='nutritional_assessment']
new_label_df = pd.concat([new_label_df, pr_df, pq_df, dd_df, ca_df, na_df], axis=0, ignore_index=True)

In [7]:
# Check the distribution and see if it satisfies the 10:1 ratio
new_label_df['class'].value_counts()

procedures                3535
discharge_disposition     1266
case_management           1048
physician_query            932
nutritional_assessment     763
Name: class, dtype: int64

In [8]:
# For consistency, rename the new_labe_df back to label_df
label_df = new_label_df

In [11]:
# Drop duplicate records
label_df.drop_duplicates(inplace=True)
label_df

page                   class                          filename  \
0        7              procedures  1003700000000633142_11068501.pdf   
1       76              procedures  1003700000000633142_11068501.pdf   
2      100              procedures  1003700000000687853_11082372.pdf   
3       18              procedures  1003700000000674647_11082141.pdf   
4       19              procedures  1003700000000674647_11082141.pdf   
...    ...                     ...                               ...   
7539   279  nutritional_assessment  1003700000000634782_11248114.pdf   
7540   280  nutritional_assessment  1003700000000634782_11248114.pdf   
7541    57  nutritional_assessment  1003700000000648050_11248145.pdf   
7542    58  nutritional_assessment  1003700000000648050_11248145.pdf   
7543    59  nutritional_assessment  1003700000000648050_11248145.pdf   

      filename_cut  
0           633142  
1           633142  
2           687853  
3           674647  
4           674647  
...            ...  
7539        634782  
7540        634782  
7541        648050  
7542        648050  
7543        648050  

[7535 rows x 4 columns]

In [12]:
# Check common file
common_files = set(textract_df.filename).intersection(label_df.filename)
common_files_train = common_files
len(common_files_train)

1498

In [13]:
# filter out uncommon file in text dataframe
textract_df_train = textract_df[textract_df.filename.isin(common_files_train)]
textract_df_train

filename  page  \
122     1003700000000543962_11116157.pdf     1   
123     1003700000000543962_11116157.pdf     2   
124     1003700000000543962_11116157.pdf     3   
125     1003700000000543962_11116157.pdf     4   
126     1003700000000543962_11116157.pdf     5   
...                                  ...   ...   
840163  1003700000000697103_11103804.pdf    43   
840164  1003700000000697103_11103804.pdf    44   
840165  1003700000000697103_11103804.pdf    45   
840166  1003700000000697103_11103804.pdf    46   
840167  1003700000000697103_11103804.pdf    47   

                                                  content  
122     HMRM22998313Member/Patient JOHN P TADLOCK:Serv...  
123     8F-TRANSPLANTTadlock, John PaulTGH1 Tampa Gene...  
124     8F-TRANSPLANTTadlock, John PaulTGH1 Tampa Gene...  
125     8F-TRANSPLANTTadlock, John PaulTG1 Tampa Gener...  
126     8F-TRANSPLANTTadlock, John PaulTGH1 Tampa Gene...  
...                                                   ...  
840163  Patient: GABRY, KAREN11800 E. Twelve Mile Road...  
840164  Patient:GABRY, KAREN11800 E. Twelve Mile RoadA...  
840165  Patient:GABRY, KAREN11800 E. Twelve Mile RoadA...  
840166  Patient:GABRY, KAREN11800 E. Twelve Mile RoadA...  
840167  Patient:GABRY, KAREN11800 E. Twelve Mile RoadA...  

[577294 rows x 3 columns]

In [14]:
# filter out uncommon file in label dataframe
label_df_train = label_df[label_df.filename.isin(common_files_train)]
label_df_train

page                   class                          filename  \
0        7              procedures  1003700000000633142_11068501.pdf   
1       76              procedures  1003700000000633142_11068501.pdf   
2      100              procedures  1003700000000687853_11082372.pdf   
3       18              procedures  1003700000000674647_11082141.pdf   
4       19              procedures  1003700000000674647_11082141.pdf   
...    ...                     ...                               ...   
7539   279  nutritional_assessment  1003700000000634782_11248114.pdf   
7540   280  nutritional_assessment  1003700000000634782_11248114.pdf   
7541    57  nutritional_assessment  1003700000000648050_11248145.pdf   
7542    58  nutritional_assessment  1003700000000648050_11248145.pdf   
7543    59  nutritional_assessment  1003700000000648050_11248145.pdf   

      filename_cut  
0           633142  
1           633142  
2           687853  
3           674647  
4           674647  
...            ...  
7539        634782  
7540        634782  
7541        648050  
7542        648050  
7543        648050  

[7522 rows x 4 columns]

In [15]:
# merge text files with labels
merged_file_train = pd.merge(textract_df_train, label_df_train, how = "left", on = ["filename", "page"])
merged_file_train

filename  page  \
0       1003700000000543962_11116157.pdf     1   
1       1003700000000543962_11116157.pdf     2   
2       1003700000000543962_11116157.pdf     3   
3       1003700000000543962_11116157.pdf     4   
4       1003700000000543962_11116157.pdf     5   
...                                  ...   ...   
577428  1003700000000697103_11103804.pdf    43   
577429  1003700000000697103_11103804.pdf    44   
577430  1003700000000697103_11103804.pdf    45   
577431  1003700000000697103_11103804.pdf    46   
577432  1003700000000697103_11103804.pdf    47   

                                                  content class  filename_cut  
0       HMRM22998313Member/Patient JOHN P TADLOCK:Serv...   NaN           NaN  
1       8F-TRANSPLANTTadlock, John PaulTGH1 Tampa Gene...   NaN           NaN  
2       8F-TRANSPLANTTadlock, John PaulTGH1 Tampa Gene...   NaN           NaN  
3       8F-TRANSPLANTTadlock, John PaulTG1 Tampa Gener...   NaN           NaN  
4       8F-TRANSPLANTTadlock, John PaulTGH1 Tampa Gene...   NaN           NaN  
...                                                   ...   ...           ...  
577428  Patient: GABRY, KAREN11800 E. Twelve Mile Road...   NaN           NaN  
577429  Patient:GABRY, KAREN11800 E. Twelve Mile RoadA...   NaN           NaN  
577430  Patient:GABRY, KAREN11800 E. Twelve Mile RoadA...   NaN           NaN  
577431  Patient:GABRY, KAREN11800 E. Twelve Mile RoadA...   NaN           NaN  
577432  Patient:GABRY, KAREN11800 E. Twelve Mile RoadA...   NaN           NaN  

[577433 rows x 5 columns]

In [16]:
# Drop unlabelled records as we already removed "others" class in first step
merged_file_train.dropna(subset=['class'], inplace=True)
merged_file_train

filename  page  \
81      1003700000000543962_11116157.pdf    82   
82      1003700000000543962_11116157.pdf    83   
83      1003700000000543962_11116157.pdf    84   
84      1003700000000543962_11116157.pdf    85   
85      1003700000000543962_11116157.pdf    86   
...                                  ...   ...   
576951  1003700000000746744_11238808.pdf    42   
576957  1003700000000746744_11238808.pdf    48   
576960  1003700000000746744_11238808.pdf    51   
577292  1003700000000697369_11103809.pdf    13   
577394  1003700000000697103_11103804.pdf     9   

                                                  content  \
81      8F-TRANSPLANTTadlock, John PaulTGH1 Tampa Gene...   
82      8F-TRANSPLANTTadlock, John PaulTGH1 Tampa Gene...   
83      8F-TRANSPLANTTadlock, John PaulTGH1 Tampa Gene...   
84      8F-TRANSPLANTTadlock, John PaulTG1 Tampa Gener...   
85      8F-TRANSPLANTTadlock, John PaulTG1 Tampa Gener...   
...                                                   ...   
576951  Carrollton Medical CenterLong, Buford210317515...   
576957  Carrollton Medical CenterLong, Buford30117MRN:...   
576960  Carrollton Medical CenterLong, Buford301172103...   
577292  Smith, Logan A2650 RIDGE AVENUEMRN: 216027151,...   
577394  Patient: GABRY, KAREN11800 E. Twelve Mile Road...   

                        class  filename_cut  
81                 procedures      543962.0  
82                 procedures      543962.0  
83                 procedures      543962.0  
84                 procedures      543962.0  
85                 procedures      543962.0  
...                       ...           ...  
576951  discharge_disposition      746744.0  
576957        case_management      746744.0  
576960        case_management      746744.0  
577292             procedures      697369.0  
577394             procedures      697103.0  

[7560 rows x 5 columns]

In [17]:
# Prepare data ready for "multi-label" mode
merged_file_train = merged_file_train[['content', 'class']].drop_duplicates().groupby('content')['class'].apply('|'.join).reset_index()

In [19]:
# Add most recent keyword records (prepared in other notebooks)
keyword_file = "keywords_to_training_v2.csv"
keyword_df = pd.read_csv(keyword_file)
keyword_df

class                      content
0                admit_type  Admission status: Inpatient
1                admit_type    admission for observation
2                admit_type             admission orders
3                admit_type   admission status inpatient
4                admit_type       admission to inpatient
..                      ...                          ...
700  nutritional_assessment         Nutritional Problems
701  nutritional_assessment    nutritional re-assessment
702  nutritional_assessment        Reason for Assessment
703  nutritional_assessment     Service  Type: Nutrition
704  nutritional_assessment           service:nutrtition

[705 rows x 2 columns]

In [ ]:
# In the keyword file, only keeping the records for PR/PQ/DD/CM/NA classes
new_keyword_df = pd.DataFrame()
pr_df = keyword_df[keyword_df['class']=='procedures']
pq_df = keyword_df[keyword_df['class']=='physician_query']
dd_df = keyword_df[keyword_df['class']=='discharge_disposition']
ca_df = keyword_df[keyword_df['class']=='case_management']
na_df = keyword_df[keyword_df['class']=='nutritional_assessment']
new_keyword_df = pd.concat([new_keyword_df, pr_df, pq_df, dd_df, ca_df, na_df], axis=0, ignore_index=True)

In [22]:
# Concatenate training records with keyword records
merged_file_train = pd.concat([merged_file_train, new_keyword_df], axis=0, ignore_index = True)
merged_file_train

content  \
0     "Interval recanalization of the right iliofemo...   
1     # 932 2,053/121210517501768OLYMPIAMEDICAL CENT...   
2     ## START~0576648~51217961~CDIQ~##END ~HMC VRF ...   
3     ##START~1356072~51260774~ASMTCARE~##ENDHMC VRF...   
4     ##START~1356072~51260774~ASMTCARE~##ENDHMC VRF...   
...                                                 ...   
7600                               Nutritional Problems   
7601                          nutritional re-assessment   
7602                              Reason for Assessment   
7603                           Service  Type: Nutrition   
7604                                 service:nutrtition   

                       class  
0     nutritional_assessment  
1                 procedures  
2            physician_query  
3     nutritional_assessment  
4     nutritional_assessment  
...                      ...  
7600  nutritional_assessment  
7601  nutritional_assessment  
7602  nutritional_assessment  
7603  nutritional_assessment  
7604  nutritional_assessment  

[7605 rows x 2 columns]

In [23]:
# Reorder the training dataframe to satisfy Comprehend's requirement
merged_file_train = merged_file_train[['class', 'content']]

In [24]:
# Save training data locally
merged_file_train.to_csv('multistep-3-lowergroup-training-data-autosplit-keyword.csv', index=False, header=False)

In [25]:
# Specify region and client 
REGION = 'us-east-1'
s3 = boto3.resource('s3')
comprehend = boto3.client('comprehend', region_name = REGION)

In [26]:
# upload training file to S3 (we're uploading to a folder that training pipeline is not monitoring, so we can manually kickoff jobs)
s3.Bucket('scio-saas-datastores').upload_file('multistep-3-lowergroup-training-data-autosplit-keyword.csv', 'SaaS_workspace/aws_comprehend/inputs_manual/multistep-3-lowergroup-training-data-autosplit-keyword.csv')

In [27]:
# Specify training parameters (classifier name has to be UNIQUE)
name = 'multistep-3b-custom-classifier-lowergroup-yy-autosplit-keyword'
language_code = 'en'
training_bucket = 'scio-saas-datastores'
training_key = 'SaaS_workspace/aws_comprehend/inputs_manual/multistep-3-lowergroup-training-data-autosplit-keyword.csv'
data_access_role_arn = 'arn:aws:iam::097079433570:role/service-role/AmazonComprehendServiceRole-aws_scio_S3Datastore_servicewrite'

In [28]:
# Kick off training job
response = comprehend.create_document_classifier(
        DocumentClassifierName = name,
        LanguageCode = language_code,
        InputDataConfig = {
            "DataFormat": "COMPREHEND_CSV",
            "S3Uri":f"s3://{training_bucket}/{training_key}"
        },

        OutputDataConfig = {
            "S3Uri":"s3://scio-saas-datastores/SaaS_workspace/aws_comprehend/output/batch3-multistep",
            "KmsKeyId": "arn:aws:kms:us-east-1:097079433570:key/1c6d057c-0be3-40a6-b23c-5870a92b901f"
        },

        DataAccessRoleArn = data_access_role_arn,
        VolumeKmsKeyId = "arn:aws:kms:us-east-1:097079433570:key/1c6d057c-0be3-40a6-b23c-5870a92b901f",
        Mode = "MULTI_LABEL",

)